In [39]:
import numpy as np
from scipy.linalg import solve

In [40]:
np.random.seed(0)

### 3. Next step solution

In [63]:
def interior_point_step(A, b, x, y, s, mu):
    n = x.shape[0] 
    e = np.ones_like(x)

    X = np.diag(x)
    S_inv = np.diag(1 / s)

    mu_prime = (1 - (1 / (6 * np.sqrt(n)))) * mu

    M = A @ S_inv @ X @ A.T
    rhs = b - mu_prime * A @ S_inv @ e
    dy = np.linalg.solve(M, rhs) 

    ds = -A.T @ dy
    dx = -X @ S_inv @ ds + mu_prime * S_inv @ e - x

    x_prime = x + dx
    y_prime = y + dy
    s_prime = s + ds

    return x_prime, y_prime, s_prime, mu_prime

### 9. Speed up

In [64]:
def mehrotra_step(A, b, c, x, y, s, mu, tol=1e-10):
   
    n = A.shape[1]
    e = np.ones(n)

    r_b = A @ x - b                    
    r_c = A.T @ y + s - c         

    S_inv = 1.0 / s
    X = np.diag(x)
    XS_inv = X @ S_inv         

    M = (A * XS_inv[np.newaxis, :]) @ A.T

    rhs3_aff = -(x * s)        

    tmp_aff = S_inv * rhs3_aff + XS_inv * r_c
    rhs_y_aff = -r_b - A @ tmp_aff

    dy_aff = np.linalg.solve(M, rhs_y_aff)

    At_dy_aff = A.T @ dy_aff
    dx_aff = S_inv * (x * At_dy_aff + rhs3_aff + x * r_c)  
    ds_aff = -r_c - At_dy_aff

    x_aff = x + dx_aff
    s_aff = s + ds_aff
    
    mu_aff = np.dot(x_aff, s_aff) / n
    sigma = (mu_aff / mu) ** 3

    rhs3_agg = - (x * s) - (dx_aff * ds_aff) + sigma * mu * e 

    tmp = S_inv * rhs3_agg + XS_inv * r_c
    rhs_y = -r_b - A @ tmp

    dy = np.linalg.solve(M, rhs_y)
    At_dy = A.T @ dy
    dx = S_inv * (x * At_dy + rhs3_agg + x * r_c)
    ds = -r_c - At_dy

    x_new = x + dx
    y_new = y + dy
    s_new = s + ds

    mu_new = np.dot(x_new, s_new) / n

    return x_new, y_new, s_new, mu_new


### 6. Show that vectors are strictly feasiblefor X and its dual

In [65]:
c = np.array([-3, -4, 0, 0, 0], dtype=float)

A = np.array([[3, 3, 3, 0, 0],
              [3, 1, 0, 1, 0],
              [1, 4, 0, 0, 1]], dtype=float)

b = np.array([4, 3, 4], dtype=float )

In [66]:
x = np.array([2/5, 8/15, 2/5, 19/15, 22/15], dtype=float)
y = np.array([-4/5, -4/5, -2/3], dtype=float)
s = np.array([37/15, 28/15, 12/5, 4/5, 2/3], dtype=float)

In [45]:
# Primal
#  x > 0
print("X is strictily positive:", all(x > 0)) 

# Ax = b
print("Ax = b:", np.allclose(A @ x, b, atol=1e-10))

# Dual
# s > 0
print("s is strictly positive:", all(s > 0))

# A^T y + s = c
print("A^T y + s = c:", np.allclose(A.T @ y + s, c, atol=1e-10))

print("x, y and s are strictly feasible solutions of both problem X and its dual")

X is strictily positive: True
Ax = b: True
s is strictly positive: True
A^T y + s = c: True
x, y and s are strictly feasible solutions of both problem X and its dual


### 7. Show x,y,s are good starting solution for P_mu

In [46]:
n = x.shape[0]
mu = (x @ s) / n
mu

np.float64(0.9866666666666667)

In [47]:
# Complementary slackness
for xi, si in zip(x, s):
    print(f"x_i * s_i = {xi * si:.4f} (mu = {mu:.4f})")
print("The vectors are a good starting solution")
print("Since mu is close to 1, the choice for 1 would be a good initial guess.")

x_i * s_i = 0.9867 (mu = 0.9867)
x_i * s_i = 0.9956 (mu = 0.9867)
x_i * s_i = 0.9600 (mu = 0.9867)
x_i * s_i = 1.0133 (mu = 0.9867)
x_i * s_i = 0.9778 (mu = 0.9867)
The vectors are a good starting solution
Since mu is close to 1, the choice for 1 would be a good initial guess.


In [48]:
for xi, si in zip(x, s):
    print((xi * si - mu) / mu)
gap = float(c @ x - b @ y)
gap

1.1252260384714425e-16
0.009009009009008953
-0.027027027027027088
0.027027027027027088
-0.009009009009009179


4.933333333333334

### 10. Stop heuristics 

In [ ]:
def verify(A, b, c, x, y, s, tol_fix=1e-10, tol=1e-14):
    n = len(x)
    m = A.shape[0]

    fix_x = np.zeros(n, dtype=bool)

    for i in range(n):
        if x[i] * s[i] < tol_fix:
            if abs(x[i]) < abs(s[i]):
                fix_x[i] = True

    free_x_idx = np.where(~fix_x)[0]

    A_r = A[:, free_x_idx]
    c_r = c[free_x_idx]

    y_sol, _, _, _ = np.linalg.lstsq(A_r.T, c_r, rcond=None)

    x_r, _, _, _ = np.linalg.lstsq(A_r, b, rcond=None)

    s_r = c_r - A_r.T @ y_sol

    x_new = np.zeros_like(x)
    s_new = np.zeros_like(s)
    y_new = y_sol

    x_new[free_x_idx] = x_r
    s_new[free_x_idx] = s_r

    primal_res = np.linalg.norm(A @ x_new - b)
    dual_res = np.linalg.norm(A.T @ y_new + s_new - c)

    nonneg = np.all(x_new > -tol) and np.all(s_new > -tol)

    feasible = (primal_res < tol) and (dual_res < tol) and nonneg 
    if feasible:
        return True, x_new, y_new, s_new
    else:
        return False, None, None, None


In [ ]:

def exact_from_zero_sets(A, b, c, x, y, s):
    n = A.shape[1]; m = A.shape[0]

    X0, S0 = [], []
    for i in range(n):
        if x[i] <= s[i]:
            X0.append(i)
        else:
            S0.append(i)

    M = np.zeros((m + n + len(X0) + len(S0), 2*n + m))
    rhs = np.zeros(M.shape[0])

    r = 0
    # Ax = b
    M[r:r+m, :n] = A; rhs[r:r+m] = b; r += m
    # A^T y + s = c
    M[r:r+n, n:n+m] = A.T
    
    for i in range(n): 
        M[r+i, n+m+i] = 1.0
    
    rhs[r:r+n] = c; r += n

    for i in X0: 
        M[r, i] = 1.0; r += 1
    
    for i in S0: 
        M[r, n+m+i] = 1.0; r += 1

    try:
        xys = np.linalg.solve(M, rhs)
    except:
        return False, False, False, False   # Not optimal if we get a singular matrix
    
    x = xys[:n]; y = xys[n:n+m]; s = xys[n+m:]

    primal = np.allclose(A@x, b, atol=1e-10)
    dual   = np.allclose(A.T@y + s, c, atol=1e-10)
    nonneg = np.all(x >= -1e-12) and np.all(s >= -1e-12)
    comp   = np.all(np.abs(x*s) <= 1e-10)
    p_obj  = float(c @ x); d_obj = float(b @ y)
    sd_ok  = abs(p_obj - d_obj) <= 1e-9

    optimal = primal and dual and nonneg and comp and sd_ok

    return x, y, s, optimal


### 8. Iterate next-step to convergence

In [130]:
count = 0

_x, _y, _s, _mu = x.copy(), y.copy(), s.copy(), mu
while True:
    x_prime, y_prime, s_prime, mu_prime = interior_point_step(A, b, _x, _y, _s, _mu)
    x_fixed, y_fixed, s_fixed, optimal = exact_from_zero_sets(A, b, c, x_prime, y_prime, s_prime)
    
    
    if optimal:
        print("Optimal")
        _x, _y, _s = x_fixed, y_fixed, s_fixed
        break
    
    count += 1

    if np.allclose(_x, x_prime, atol=1e-20) and np.allclose(_y, y_prime, atol=1e-20) and np.allclose(_s, s_prime, atol=1e-20):
        break
    
    _x, _y, _s, _mu = x_prime, y_prime, s_prime, mu_prime
    
print("x", _x)
print("y", _y)
print("s", _s)
print("mu", _mu)
print(f"Converged in {count} iterations")

Optimal
x [ 0.44444444  0.88888889 -0.          0.77777778  0.        ]
y [-8.88888889e-01  7.40148683e-17 -3.33333333e-01]
s [0.         0.         2.66666667 0.         0.33333333]
mu 0.20958722618285858
Converged in 20 iterations


In [104]:
from fractions import Fraction

def to_fraction(x):
    return Fraction(x).limit_denominator()

x_ = [to_fraction(xi) for xi in _x]
x_

[Fraction(329, 492),
 Fraction(97, 123),
 Fraction(-61, 492),
 Fraction(101, 492),
 Fraction(29, 164)]

In [99]:
print(4/9)

0.4444444444444444


In [100]:
print("Test if the solution is truly exact:")
print(A @ _x, b)
print(A.T @ _y + _s, c)

Test if the solution is truly exact:
[4. 3. 4.] [4. 3. 4.]
[-3. -4.  0.  0.  0.] [-3. -4.  0.  0.  0.]


In [52]:
count = 0

_x, _y, _s, _mu = x.copy(), y.copy(), s.copy(), mu
while True:
    x_prime, y_prime, s_prime, mu_prime = mehrotra_step(A, b, c, _x, _y, _s, _mu, tol=1e-15)
    feasible, x_fixed, y_fixed, s_fixed = verify(A, b, c, x_prime, y_prime, s_prime)
    
    if feasible:
        print("Optimal")
        _x, _y, _s = x_fixed, y_fixed, s_fixed
        break
    
    count += 1

    if np.allclose(_x, x_prime, atol=1e-15) and np.allclose(_y, y_prime, atol=1e-15) and np.allclose(_s, s_prime, atol=1e-15):
        break
    
    _x, _y, _s, _mu = x_prime, y_prime, s_prime, mu_prime
    
print("x", _x)
print("y", _y)
print("s", _s)
print("mu", _mu)
print(f"Converged in {count} iterations")

x [ 4.44444444e-01  8.88888889e-01 -2.28301849e-22  7.77777778e-01
  5.64335701e-20]
y [-8.88888889e-01  2.45639555e-20 -3.33333333e-01]
s [ 2.77932686e-22  8.99972506e-22  2.66666667e+00 -2.45639555e-20
  3.33333333e-01]
mu 4.117521271365709e-24
Converged in 5 iterations


In [53]:
count = 0

_x, _y, _s, _mu = x.copy(), y.copy(), s.copy(), mu

while True:
    x_prime, y_prime, s_prime, mu_prime = interior_point_step(A, b, _x, _y, _s, _mu)
    count += 1

    if np.allclose(_x, x_prime, atol=1e-20) and np.allclose(_y, y_prime, atol=1e-20) and np.allclose(_s, s_prime, atol=1e-20):
        break
    
    _x, _y, _s, _mu = x_prime, y_prime, s_prime, mu_prime
    
print("x", _x)
print("y", _y)
print("s", _s)
print("mu", _mu) 
print(f"Converged in {count} iterations")

x [4.44444444e-01 8.88888889e-01 1.55381428e-20 7.77777778e-01
 1.24305142e-19]
y [-8.88888889e-01 -5.32736324e-20 -3.33333333e-01]
s [9.32288568e-20 4.66144284e-20 2.66666667e+00 5.32736324e-20
 3.33333333e-01]
mu 4.143504744828719e-20
Converged in 577 iterations


In [60]:
# Check convergence
for xi, si in zip(_x, _s):
    print(f"x_i * s_i = {xi * si:.21f} (should be close to mu = {_mu:.21f})")

x_i * s_i = 0.000000000000000000041 (should be close to mu = 0.000000000000000000041)
x_i * s_i = 0.000000000000000000041 (should be close to mu = 0.000000000000000000041)
x_i * s_i = 0.000000000000000000041 (should be close to mu = 0.000000000000000000041)
x_i * s_i = 0.000000000000000000041 (should be close to mu = 0.000000000000000000041)
x_i * s_i = 0.000000000000000000041 (should be close to mu = 0.000000000000000000041)


In [62]:
print("Solution not exact without the fix:")
print(A @ _x, b)
print(A.T @ _y + _s, c)

Solution not exact without the fix:
[4. 3. 4.] [4. 3. 4.]
[-3.00000000e+00 -4.00000000e+00  2.66453526e-15  0.00000000e+00
  0.00000000e+00] [-3. -4.  0.  0.  0.]


### 12. Commercial solver: simplex vs. interior point

In [81]:
from scipy.optimize import linprog
import warnings

warnings.filterwarnings("ignore", category=DeprecationWarning)

In [83]:
sol_simplex = linprog(c, A_eq=A, b_eq=b, method='simplex')
sol_simplex.x

array([0.44444444, 0.88888889, 0.        , 0.77777778, 0.        ])

In [84]:
sol_ip = linprog(c, A_eq=A, b_eq=b, method='interior-point')
sol_ip.x

array([4.44444444e-01, 8.88888889e-01, 2.08281533e-13, 7.77777778e-01,
       1.89801273e-14])